In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
import numpy as np
from sqlalchemy import create_engine
import ipywidgets as widgets
from IPython.display import display, clear_output

db_name = "timescale"
db_user = "admin"
db_password = "TimescalePassword"
db_host = "localhost"
db_port = "5433"
database_uri = f"postgresql://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}"
engine = create_engine(database_uri)
df2 = pd.read_sql("SELECT * FROM filteredsensordatagrid", engine)

sections = ["Vihannekset", "Hedelmät", "Viljatuotteet", "Einekset", "Lihatuotteet", "Kuivatuotteet", "Välipalat", "Virvokkeet", "Maitotuotteet", "WC-paperit", "Pesuaineet", "Hygieniatuotteet", "Makeiset", "Pakasteet", "Lemmikkien ruuat", "Sisäänkäynti", "Pulloautomaatti"]
weekdays = [0, 1, 2, 3, 4, 5, 6]  # 0 = maanantai, 6 = sunnuntai
ajat = list(range(24))  # 0-23

dropdown = widgets.Dropdown(options=sections, description='osasto:')
pv_dropdown = widgets.Dropdown(options=weekdays, description='viikonpäivä:')
time_dropdown = widgets.Dropdown(options=ajat, description='kellonaika:')
nappi = widgets.Button(description='laske tn')
output = widgets.Output()

def tn(section, weekday, time):
    filtered_df = df2[(df2['viikonpaiva'] == weekday) & (df2['tunti'] == time)]         # suodatetaan dataframesta viikonpäivä ja kellonaika
    reitit_total = filtered_df.shape[0]                                                 # kauppareissujen määrä
    reitit = filtered_df[filtered_df[section] == 1].shape[0]                            # kuinka monta reissua käy osastolla X, (sarakkeet joissa arvo on 1)
    probability = reitit / reitit_total if reitit_total > 0 else 0                      # # osastolla X käyneiden reissujen määrä / reissujen määrä
    return probability

def nappi_clicked(b):
    with output:
        clear_output(wait=True)
        section = dropdown.value
        weekday = pv_dropdown.value
        time = time_dropdown.value
        probability = tn(section, weekday, time)
        print(f"[ {probability:.2f} ] kauppareissuista ovat käyneet osastolla {section}, viikonpäivänä {weekday} klo {time}: ")

nappi.on_click(nappi_clicked)
display(dropdown, pv_dropdown, time_dropdown, nappi, output)


Dropdown(description='osasto:', options=('Vihannekset', 'Hedelmät', 'Viljatuotteet', 'Einekset', 'Lihatuotteet…

Dropdown(description='viikonpäivä:', options=(0, 1, 2, 3, 4, 5, 6), value=0)

Dropdown(description='kellonaika:', options=(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18,…

Button(description='laske tn', style=ButtonStyle())

Output()